In [ ]:
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re
from tensorflow.keras.utils import to_categorical

data = open('/content/thotb.txt', "r")

nltk.download('punkt')
cleaned = re.sub(r'\W+', ' ', data.read()).lower()
tokens = word_tokenize(cleaned)
train_len = 4
text_sequences = []
for i in range(train_len,len(tokens)):
  seq = tokens[i-train_len:i]
  text_sequences.append(seq)
sequences = {}
count = 1
for i in range(len(tokens)):
  if tokens[i] not in sequences:
    sequences[tokens[i]] = count
    count += 1
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)
#vocabulary size increased by 1 for the cause of padding
vocabulary_size = len(tokenizer.word_counts)+1
n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
  n_sequences[i] = sequences[i]
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Embedding
model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(SimpleRNN(50,return_sequences=True))
model.add(SimpleRNN(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 3)              16695     
                                                                 
 simple_rnn (SimpleRNN)      (None, 3, 50)             2700      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 50)                5050      
                                                                 
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 5565)              283815    
                                                                 
Total params: 310,810
Trainable params: 310,810
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# compiling the network
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(train_inputs,train_targets,epochs=20,verbose=1)

Epoch 1/20
1875/1875 [==============================] - 21s 10ms/step - loss: 6.3453
Epoch 2/20
1875/1875 [==============================] - 19s 10ms/step - loss: 5.8138
Epoch 3/20
1875/1875 [==============================] - 19s 10ms/step - loss: 5.5242
Epoch 4/20
1875/1875 [==============================] - 18s 10ms/step - loss: 5.3350
Epoch 5/20
1875/1875 [==============================] - 18s 10ms/step - loss: 5.1807
Epoch 6/20
1875/1875 [==============================] - 19s 10ms/step - loss: 5.0433
Epoch 7/20
1875/1875 [==============================] - 19s 10ms/step - loss: 4.9200
Epoch 8/20
1875/1875 [==============================] - 19s 10ms/step - loss: 4.8148
Epoch 9/20
1875/1875 [==============================] - 19s 10ms/step - loss: 4.7219
Epoch 10/20
1875/1875 [==============================] - 19s 10ms/step - loss: 4.6370
Epoch 11/20
1875/1875 [==============================] - 18s 10ms/step - loss: 4.5603
Epoch 12/20
1875/1875 [==============================] - 19s 10

In [ ]:
from keras.preprocessing.sequence import pad_sequences
arr = ['i want to', 'can you', 'upon the', 'the victim', 'as far']
for teks in arr:
  input_text = teks.strip()
  print("Input text = ",input_text)
  encoded_text = tokenizer.texts_to_sequences([input_text])[0]
  pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
  for j in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
    pred_word = tokenizer.index_word[j]
    print("Next word suggestion:",pred_word)
  print("=================================================")

Input text =  i want to
Next word suggestion: be
Next word suggestion: me
Next word suggestion: the
Input text =  can you
Next word suggestion: have
Next word suggestion: know
Next word suggestion: be
Input text =  upon the
Next word suggestion: moor
Next word suggestion: right
Next word suggestion: contrary
Input text =  the victim
Next word suggestion: of
Next word suggestion: to
Next word suggestion: and
Input text =  as far
Next word suggestion: to
Next word suggestion: in
Next word suggestion: as


In [ ]:
input_text = 'the most serious problem'.strip()
for i in range (9):
  print("Input text = ",input_text)
  encoded_text = tokenizer.texts_to_sequences([input_text])[0]
  pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
  for j in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
    print("Next word suggestion:",tokenizer.index_word[j])
  pred_word = tokenizer.index_word[(model.predict(pad_encoded)[0]).argsort()[-3:][::-1][0]]
  print("=================================================")
  input_text = input_text+' '+pred_word

Input text =  the most serious problem
Next word suggestion: and
Next word suggestion: in
Next word suggestion: i
Input text =  the most serious problem and
Next word suggestion: i
Next word suggestion: the
Next word suggestion: he
Input text =  the most serious problem and i
Next word suggestion: have
Next word suggestion: had
Next word suggestion: could
Input text =  the most serious problem and i have
Next word suggestion: been
Next word suggestion: not
Next word suggestion: seen
Input text =  the most serious problem and i have been
Next word suggestion: a
Next word suggestion: able
Next word suggestion: so
Input text =  the most serious problem and i have been a
Next word suggestion: fixture
Next word suggestion: man
Next word suggestion: school
Input text =  the most serious problem and i have been a fixture
Next word suggestion: he
Next word suggestion: that
Next word suggestion: i
Input text =  the most serious problem and i have been a fixture he
Next word suggestion: had
Next